In [1]:
! pip install ultralytics

In [2]:
! pip install tensorflow

In [21]:
from ultralytics import YOLO
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# Load YOLO object detection model
yolo_model = YOLO("best.pt")

# Load CNN classification model
cnn_model = load_model("cnn.keras")

# Load image
image_path = "Sikkimese woman in traditional dress_Image_34.jpg"
image = cv2.imread(image_path)

In [22]:
di = {
    'AndhraPradeshDressingStyle': 0,
    'JammuKashmirDressingStyle': 1,
    'KeralaDressingStyle': 2,
    'MaharashtraDressingStyle': 3,
    'SikkimDressingStyle': 4
}

# Reverse mapping: index to class name
idx_to_label = {v: k for k, v in di.items()}

# Run object detection
results = yolo_model(image)[0]  # First image's detection result

def classify(image):
    try:
        # Normalize and prepare image
        img = image / 255.0
        img = np.expand_dims(img, axis=0)

        # CNN prediction
        prediction = cnn_model.predict(img)[0]
        return prediction

    except Exception as e:
        return f"Prediction Error: {e}"

# Loop through detected bounding boxes
for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])

    # Crop and resize
    cropped = image[y1:y2, x1:x2]
    cropped_resized = cv2.resize(cropped, (256, 256))

    # Predict
    prediction = classify(cropped_resized)
    class_id = np.argmax(prediction)
    confidence = prediction[class_id]

    # Map class_id to label
    class_name = idx_to_label[class_id]

    print(f"Predicted Class: {class_name}, Confidence: {confidence:.2f}")

    # Annotate image
    label = f"{class_name} ({confidence:.2f})"
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, label, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

# Display image (if running locally)
cv2.imshow("Result", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 640x448 1 Style, 152.0ms
Speed: 5.3ms preprocess, 152.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 448)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
Predicted Class: AndhraPradeshDressingStyle, Confidence: 0.90


In [ ]:
di = {'AndhraPradeshDressingStyle': 0,
 'JammuKashmirDressingStyle': 1,
 'KeralaDressingStyle': 2,
 'MaharashtraDressingStyle': 3,
 'SikkimDressingStyle': 4}